In [71]:
SAMPLE = False

#  Lexical Resource (LR) for Annotation

#### Imports

In [72]:
# interface
from tqdm import tqdm
from dsutils.de.files import dbg

## Files and filesystem
import os
import json
import csv
import glob

## Data management
import numpy as np
import pandas as pd

## DS tools
import re
from types import NoneType
from dsutils.de.files import describe_csv, get_csv_head, xls_to_csv, get_data_path, get_datafile_path
from dsutils.nlp.language import get_lang_code

#### Paths

In [73]:
## input
db_paths = dict(
    eppo_com = get_datafile_path('original/2022-09-02_COMMONnames_EPPO_OQ.csv'),
    eppo_sci = get_datafile_path('original/2022-09-02_SCIENTIFICnames_EPPO_OQ.csv'),
    efsa = get_datafile_path('EFSA-keyword-match/FichierMotsClesMagaliLarenaudie.csv'),
    ncbi = get_datafile_path('taxa+id_full.txt'), # or taxa+id_microbes+insects.txt
)

glossary_paths = dict(
    efsa_glossary = get_datafile_path('efsa_glossary.csv'),
    eppo_glossary = get_datafile_path('eppo_glossary.csv'),
)

## Output
data_path = get_data_path()
full_eppo_path = os.path.join(data_path, 'eppo_glossary.csv')
annotation_path = os.path.join(data_path, 'LRs_for_annotation')
anno_gloss_path = os.path.join(data_path, 'annotation_glossary.csv')
LR_filepath = 'LR.csv'

#### Definitions

Defining functions for preprocessing of original data

In [74]:
def PHT_to_lower_taxon(PHT_code):
    taxon = re.sub(r'(?<!^)(?=[A-Z])', ' ', PHT_code[:-4]).lower()
    return taxon
# print('AcalymmaVittatum-PHT:',PHT_to_lower_taxon('AcalymmaVittatum-PHT')) # test
assert PHT_to_lower_taxon('AcalymmaVittatum-PHT') == 'acalymma vittatum'

def to_pattern(syn):
    r = syn.replace('+', ' ')
    r = r.replace('_', '\\w')
    if r.endswith('%'):
        r = r[:-1] + '\\w*'
    r = r.replace('% ', '\\w* ')
    r = r.replace('%', '\\S*\\s?')
    return r
assert to_pattern(r'chrysom_le%+ray_e+du+concombre') == 'chrysom\wle\w* ray\we du concombre'

#### Parameters

In [75]:
ncbi_sep = '\t'
ncbi_header = ['synonym','taxid','sci_name','tax_path','POS','rank','_', '__']
ncbi_ext = '.csv'

db_paths['ncbi'] = db_paths['ncbi'][:-4]+ncbi_ext
if SAMPLE:
    for k in db_paths.keys():
        db_paths[k] = db_paths[k][:-4]+' copy'+db_paths[k][-4:]
    LR_filepath = LR_filepath[:-4]+'_sample'+LR_filepath[-4:]

## name of the desired coluimns in the LR
anno_gloss_column_names = ['EPPO_CODE', 'EFSA_PHT', 'NCBI_TAXID', # 3 partial keys to relative dataset
                           'preferred_name', 'synonym', 're',# terms
                           'ds_language', 'match_language'] # languages

In [76]:
## column in each of the original datasets corresponding to the output LR column 
cc = dict(
    eppo_com = ['CodeEOPP', None, None,
                'PreferredName', 'CommonName', None,
                ['Language', get_lang_code], None], # [obj (col name), fun] tuple when information is extracted via a function
    eppo_sci = ['CodeEOPP', None, None,
                'PreferredName', 'OtherScientificNames', None,
                None, None],
    efsa = [None, 'Category (pest name)', None,
                ['Category (pest name)', PHT_to_lower_taxon], None,  ['Keywords', to_pattern],
                None, 'Unnamed: 3'],
    ncbi = [None, None, 'taxid',
            'sci_name', 'synonym', None,
            None, None],
    )

In [77]:
path =get_datafile_path('taxa+id_microbes+insects.csv')
with open(path) as f:
    line = f.readline()
    line = f.readline()

print(line)

"Euscelis Brulle, 1832",ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:33208/ncbi:6072/ncbi:33213/ncbi:33317/ncbi:1206794/ncbi:88770/ncbi:6656/ncbi:197563/ncbi:197562/ncbi:6960/ncbi:50557/ncbi:85512/ncbi:7496/ncbi:33340/ncbi:33342/ncbi:7524/ncbi:1955247/ncbi:33365/ncbi:33368/ncbi:30102/ncbi:33372/ncbi:565967/ncbi:6826,NP,genus,,



In [78]:
describe_csv(path)

,synonym,taxid,sci_name,tax_path,POS,rank,_,__
count,859017,859017,859017,859017,859017,859017,0.0,0.0
unique,810220,238405,238305,238405,3,28,NaN,NaN
top,P. affinis,ncbi:7175,Culex pipiens,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,species,NaN,NaN
freq,22,174,174,174,855514,766080,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
get_csv_head(path)

,synonym,taxid,sci_name,tax_path,POS,rank,_,__
0,"Euscelis Brulle, 1832",ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN
1,Euscelis Brulle,ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN
2,Euscelis (Brulle),ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN
3,Euscelis,ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN
4,Eusceli,ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN


In [80]:
pd.read_csv(path, on_bad_lines='skip')

,synonym,taxid,sci_name,tax_path,POS,rank,_,__
0,"Euscelis Brulle, 1832",ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN
1,Euscelis Brulle,ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN
2,Euscelis (Brulle),ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN
3,Euscelis,ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN
4,Eusceli,ncbi:6826,Euscelis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,genus,NaN,NaN
...,...,...,...,...,...,...,...,...
859012,R.errabunda,ncbi:1519426,Ravinia errabunda,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,species,NaN,NaN
859013,R . errabunda,ncbi:1519426,Ravinia errabunda,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,species,NaN,NaN
859014,"Ravinia errabunda (Wulp, 1896)",ncbi:1519426,Ravinia errabunda,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,species,NaN,NaN
859015,Ravinia errabunda Wulp,ncbi:1519426,Ravinia errabunda,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:33154/ncbi:...,NP,species,NaN,NaN


#### Check Data

In [81]:
for db_name, path in db_paths.items():
    dbg(len(describe_csv(path).columns))
    if (len(describe_csv(path).columns))<=2:
        dbg(path)
        display(describe_csv(path))
        print('starting csv format standardisation:')
        normalised_df = pd.read_csv(path, names=ncbi_header, index_col=0, on_bad_lines='skip', sep=ncbi_sep).reset_index(level=0)
        print('standardised:')
        display(normalised_df.head())
        new_path = path[:-4]+'.csv'
        
        normalised_df.to_csv(new_path, index=False,)
        db_paths[db_name] = new_path
    display(describe_csv(path))

mess: 5


,CommonName,Language,CodeEOPP,PreferredName,AuthorityPreferredName
count,1276,1276,1324,1324,1142
unique,1235,38,239,239,169
top,citrus leprosis,English,HELIZE,Helicoverpa zea,(Boddie)
freq,4,477,30,30,30


mess: 5


,OtherScientificNames,Authority,CodeEOPP,PreferredName,AuthorityPreferredName
count,616,453,664,664,502
unique,611,324,239,239,169
top,Tomato leaf curl New Delhi begomovirus,Corbett,BEMITA,Bemisia tabaci,(Gennadius)
freq,2,6,16,16,16


_27: 4


,Category (pest name),Keywords,Unnamed: 2,Unnamed: 3
count,7558,7558,21,3
unique,1168,7393,5,3
top,Cronartium-PHT,margarodes,Au/Cabi,Fr
freq,85,3,16,1


mess: 8


,synonym,taxid,sci_name,tax_path,POS,rank,undefined,also_undefined
count,1607725,1607726,1607726,1607726,1607726,1607726,1515574,1515573
unique,1474816,242820,242457,242820,3,40,180141,180140
top,mitosporic Ascomycota,ncbi:1888,Streptomyces albus,/ncbi:1/ncbi:131567/ncbi:2/ncbi:1783272/ncbi:2...,NP,species,ncbi:11320,Influenza A virus
freq,48,532,532,532,1583049,1446117,4350,4350


# Create LR

### (1) Preprocess original datasets

Extract relevant information from each dataset and store it in table form (one `csv` per original dataset)

## Display examples of input datasets
display(pd.read_csv(db_paths['eppo_sci']).describe())
pd.read_csv(db_paths['eppo_com']).describe()

In [82]:

db = dict() # create dictionary of databases

for db_name, columns in cc.items(): # loop over db names and respective relevant columns
    db[db_name] = dict()    # new database in db dictionary
    cc_zip = zip(columns, anno_gloss_column_names)  # match source-db column names to final-db column names

    for orig_col, targ_col in cc_zip: # loop over matches
        dbg(orig_col)
        if isinstance(orig_col, str): # if only column name specified, take column as is
            def fun(x):
                return x
        elif isinstance(orig_col,NoneType): # else, if column must not to be selected, continue
            continue
        else:    # else, if a preprocessing function has been specified, use it when extracting column
            fun = orig_col[1]
            orig_col = orig_col[0]
            
        # sep = tab (for ncbi)
        try:
            ds_col = pd.read_csv(db_paths[db_name], on_bad_lines='skip')[orig_col]
        except:
            ds_col = pd.read_csv(db_paths[db_name], names=ncbi_header, index_col=0, on_bad_lines='skip', sep=ncbi_sep).reset_index(level=0)[orig_col]
            
        db[db_name][targ_col] = list(map(fun, ds_col)) # new_db col = old_db col
    
    pd.DataFrame.from_dict(db[db_name]).to_csv(os.path.join(annotation_path, db_name+'_table.csv'), index=False)


orig_col: CodeEOPP
__package__: None
__package__: None
orig_col: PreferredName
orig_col: CommonName
__package__: None
orig_col: ['Language', <function get_lang_code at 0x7f732e710d30>]
__package__: None
orig_col: CodeEOPP
__package__: None
__package__: None
orig_col: PreferredName
orig_col: OtherScientificNames
__package__: None
__package__: None
__package__: None
__package__: None
orig_col: Category (pest name)
__package__: None
orig_col: ['Category (pest name)', <function PHT_to_lower_taxon at 0x7f731bd4ae60>]
__package__: None
orig_col: ['Keywords', <function to_pattern at 0x7f731bd484c0>]
__package__: None
orig_col: Unnamed: 3
__package__: None
__package__: None
orig_col: taxid
orig_col: sci_name
orig_col: synonym
__package__: None
__package__: None
__package__: None


### (2) Join Datasets

In [83]:
prep_dbs = dict()
for db_name in cc.keys():
    prep_dbs[db_name] = pd.read_csv(os.path.join(annotation_path, db_name + '_table.csv'), on_bad_lines='skip')

In [84]:
prep_dbs['ncbi'].head()

,NCBI_TAXID,preferred_name,synonym
0,ncbi:2,Bacteria,Bacteria
1,ncbi:2,Bacteria,bacteria
2,ncbi:2,Bacteria,eubacteria
3,ncbi:2,Bacteria,Monera
4,ncbi:2,Bacteria,Procaryotae


#### A concatenated version

In [85]:
len(list(prep_dbs.items()))

4

In [86]:
list(prep_dbs.items())[0][1]

,EPPO_CODE,preferred_name,synonym,ds_language
0,LIBEAF,'Candidatus Liberibacter africanus',greening of citrus,en
1,LIBEAF,'Candidatus Liberibacter africanus',greening des agrumes,fr
2,LIBEAF,'Candidatus Liberibacter africanus',virescence des agrumes,fr
3,LIBEAF,'Candidatus Liberibacter africanus',enverdecimiento de los cítricos,es
4,LIBEAM,'Candidatus Liberibacter americanus',Brazilian citrus greening,en
...,...,...,...,...
1319,PHYP64,Grapevine flavescence dorée phytoplasma,rougeau,fr
1320,PHYP64,Grapevine flavescence dorée phytoplasma,flavescencia dorada de la vid,es
1321,PHYP64,Grapevine flavescence dorée phytoplasma,flavescenza dorata de la vite,it
1322,PHYP64,Grapevine flavescence dorée phytoplasma,Фитоплазма золотистого пожелтения винограда,ru


In [87]:
conc_list = list(prep_dbs.values())
conc_list[-1] = conc_list[-1].head()

In [88]:
concat_df = pd.concat(
    conc_list,
    ignore_index=True)
concat_df.to_csv('concatenated_glossary.csv')
len(concat_df)

9551

In [89]:
concat_df.head()

,EPPO_CODE,preferred_name,synonym,ds_language,EFSA_PHT,re,match_language,NCBI_TAXID
0,LIBEAF,'Candidatus Liberibacter africanus',greening of citrus,en,NaN,NaN,NaN,NaN
1,LIBEAF,'Candidatus Liberibacter africanus',greening des agrumes,fr,NaN,NaN,NaN,NaN
2,LIBEAF,'Candidatus Liberibacter africanus',virescence des agrumes,fr,NaN,NaN,NaN,NaN
3,LIBEAF,'Candidatus Liberibacter africanus',enverdecimiento de los cítricos,es,NaN,NaN,NaN,NaN
4,LIBEAM,'Candidatus Liberibacter americanus',Brazilian citrus greening,en,NaN,NaN,NaN,NaN


#### Create empty DF with col names

In [90]:
structured_db = pd.DataFrame(columns=anno_gloss_column_names)
display(structured_db)

,EPPO_CODE,EFSA_PHT,NCBI_TAXID,preferred_name,synonym,re,ds_language,match_language


df1 = pd.DataFrame([[3,4],[5,6]], columns=['a','b'])
df2 = pd.DataFrame([[5,6],[5,6]], columns=['a','c'])
display(df1)
pd.concat([df1,df2], ignore_index=True)

#### Add EPPO info to final DB

In [91]:
db_names = list(str(x) for x in prep_dbs.keys()) # names of EPPO databases

In [92]:
# copy into a new db to which eppo information will be added 
ncbi_eppo_db = structured_db.copy()
prep_ncbi_db = prep_dbs['ncbi']

## open both eppo (sci and com) dbs and concatenate them
prep_eppo_dbs = [prep_dbs[db_name] for db_name in db_names if 'eppo' in db_name]
prep_eppo_db = pd.concat(prep_eppo_dbs, ignore_index=True)
display(prep_eppo_db.head())

,EPPO_CODE,preferred_name,synonym,ds_language
0,LIBEAF,'Candidatus Liberibacter africanus',greening of citrus,en
1,LIBEAF,'Candidatus Liberibacter africanus',greening des agrumes,fr
2,LIBEAF,'Candidatus Liberibacter africanus',virescence des agrumes,fr
3,LIBEAF,'Candidatus Liberibacter africanus',enverdecimiento de los cítricos,es
4,LIBEAM,'Candidatus Liberibacter americanus',Brazilian citrus greening,en


In [93]:
## loop over syns in EPPO db
for prep_eppo_db_i in tqdm(range(len(prep_eppo_db))):
    syn = str(prep_eppo_db['synonym'][prep_eppo_db_i]) 
    lang = prep_eppo_db['ds_language'][prep_eppo_db_i]
    key = prep_eppo_db['EPPO_CODE'][prep_eppo_db_i]
    sci_name = prep_eppo_db['preferred_name'][prep_eppo_db_i]
    
    prep_ncbi_syn_rows = prep_ncbi_db[prep_ncbi_db['synonym']==syn]
    prep_ncbi_sci_rows = prep_ncbi_db[(prep_ncbi_db['preferred_name']==sci_name) | (prep_ncbi_db['preferred_name']=="'"+sci_name+"'")]
    
    # rows in NCBI that have a matching synonym or scientific name
    prep_ncbi_matching_rows = pd.concat([prep_ncbi_syn_rows, prep_ncbi_sci_rows], ignore_index=True)
    
    taxid_col = (prep_ncbi_matching_rows['NCBI_TAXID'])
    sci_name_col = prep_ncbi_matching_rows['preferred_name']
    
    if len(prep_ncbi_matching_rows) != 0:
        taxid = taxid_col[0]
        sci_name = sci_name_col[0]
    else:
        taxid = ''
        sci_name = prep_eppo_db['preferred_name'][prep_eppo_db_i]
    
    new_eppo_row = pd.DataFrame([[taxid, sci_name, syn, lang, key]], columns=['NCBI_TAXID', 'preferred_name', 'synonym', 'ds_language', 'EPPO_CODE'])
    
    ncbi_eppo_db = pd.concat([ncbi_eppo_db, new_eppo_row], ignore_index=True)

100%|██████████| 1988/1988 [13:55<00:00,  2.38it/s]


In [94]:
ncbi_eppo_db.to_csv('output.csv')

for i in range(10):
    print('i is:'+str(i))
    for j in range(10):
        print(j)
        if j%5 == 0 and j>0:
            print("break")
            break

#### Add EFSA info to final DB

In [95]:

# copy into a new db to which eppo information will be added 
final_db = ncbi_eppo_db.copy()

## open both eppo (sci and com) dbs and concatenate them
prep_efsa_dbs = [prep_dbs[db_name] for db_name in db_names if 'efsa' in db_name]
prep_efsa_db = pd.concat(prep_efsa_dbs, ignore_index=True)
display(prep_efsa_db.head())
len(prep_efsa_db)

,EFSA_PHT,preferred_name,re,match_language
0,AcaloleptaSejuncta-PHT,acalolepta sejuncta,acalolepta sejuncta,NaN
1,AcalymmaVittatum-PHT,acalymma vittatum,acalymma vittata,NaN
2,AcalymmaVittatum-PHT,acalymma vittatum,acalymma vittatum,NaN
3,AcalymmaVittatum-PHT,acalymma vittatum,chrysom\wle\w* ray\we du concombre,Fr
4,AcalymmaVittatum-PHT,acalymma vittatum,cistela melanocephala,NaN


7558

In [96]:
match_found = False
## loop over res i n EFSA db
for prep_efsa_db_i in tqdm(range(len(prep_efsa_db))):
    for final_db_i in range(len(final_db)):
        pattern = prep_efsa_db['re'][prep_efsa_db_i]
        syn = final_db['synonym'][final_db_i]
        
        ## if re matches a syn, add: re, efsa key, language if None
        if re.match(str(pattern), str(syn)):
            final_db['re'][final_db_i] = prep_efsa_db['re'][prep_efsa_db_i]
            final_db['EFSA_PHT'][final_db_i] = prep_efsa_db['EFSA_PHT'][prep_efsa_db_i]
            final_db['match_language'][final_db_i] = prep_efsa_db['match_language'][prep_efsa_db_i]
            match_found = True
        
    ## else, if no matches, add new line = re, syn, key, language
    if not match_found:
        pattern = prep_efsa_db['re'][prep_efsa_db_i]
        #syn = prep_efsa_db['synonym'][prep_efsa_db_i]
        key = prep_efsa_db['EFSA_PHT'][prep_efsa_db_i]
        lang = prep_efsa_db['match_language'][prep_efsa_db_i]
        
        new_efsa_row = pd.DataFrame([[pattern, syn, lang, key]], columns=['re', 'synonym', 'match_language', 'EFSA_PHT'])
        
        final_db = pd.concat([final_db, new_efsa_row], ignore_index=True)
            
final_db.head()

100%|██████████| 7558/7558 [04:45<00:00, 26.50it/s]


,EPPO_CODE,EFSA_PHT,NCBI_TAXID,preferred_name,synonym,re,ds_language,match_language
0,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,,'Candidatus Liberibacter africanus',greening of citrus,greening of citrus,en,NaN
1,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,,'Candidatus Liberibacter africanus',greening des agrumes,greening des agrumes,fr,NaN
2,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,,'Candidatus Liberibacter africanus',virescence des agrumes,vir\wscence des agrumes,fr,NaN
3,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,,'Candidatus Liberibacter africanus',enverdecimiento de los cítricos,enverdecimiento de los c\wtricos,es,NaN
4,LIBEAM,NaN,,'Candidatus Liberibacter americanus',Brazilian citrus greening,NaN,en,NaN


#### add NCBI info in incomplete EFSA lines

In [122]:
## Iterate over final db lines 
for final_db_i in tqdm(range(len(final_db))):
    pattern = final_db.at[final_db_i,'re']
    # check if row has only efsa info
    if pattern and pd.isna(final_db.at[final_db_i,'NCBI_TAXID']):
        # iterate through NCBI to find an item that corresponds to pattern
        for prep_ncbi_db_i in range(len(prep_ncbi_db)):
            syn = prep_ncbi_db.at[prep_ncbi_db_i,'synonym']
            sci_name = prep_ncbi_db.at[prep_ncbi_db_i,'preferred_name']
            taxid = prep_ncbi_db.at[prep_ncbi_db_i,'NCBI_TAXID']
            if re.match(str(pattern), str(syn)):
                dbg(syn,'MATCH:'+pattern)
                final_db.at[final_db_i,'synonym'] = syn
            if re.match(str(pattern), str(syn)) or re.match(str(pattern), str(sci_name)):
                dbg(sci_name,'MATCH:'+pattern)
                final_db.at[final_db_i,'NCBI_TAXID'] = taxid
                final_db.at[final_db_i,'preferred_name'] = sci_name
                break
        print('checked all NCBI for '+pattern)
final_db.head()

 92%|█████████▏| 1989/2164 [00:40<00:03, 49.43it/s]

checked all NCBI for acalolepta sejuncta


 92%|█████████▏| 1990/2164 [01:21<00:08, 20.13it/s]

checked all NCBI for acalymma vittata


 92%|█████████▏| 1991/2164 [02:02<00:15, 10.87it/s]

...

100%|█████████▉| 2163/2164 [1:46:54<00:35, 35.38s/it]

checked all NCBI for polilla plana del coco


100%|██████████| 2164/2164 [1:47:29<00:00,  2.98s/it]

checked all NCBI for small coconut leaf moth\w*


,EPPO_CODE,EFSA_PHT,NCBI_TAXID,preferred_name,synonym,re,ds_language,match_language
0,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,,'Candidatus Liberibacter africanus',greening of citrus,greening of citrus,en,NaN
1,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,,'Candidatus Liberibacter africanus',greening des agrumes,greening des agrumes,fr,NaN
2,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,,'Candidatus Liberibacter africanus',virescence des agrumes,vir\wscence des agrumes,fr,NaN
3,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,,'Candidatus Liberibacter africanus',enverdecimiento de los cítricos,enverdecimiento de los c\wtricos,es,NaN
4,LIBEAM,NaN,,'Candidatus Liberibacter americanus',Brazilian citrus greening,NaN,en,NaN
...,...,...,...,...,...,...,...,...
2159,NaN,AgonoxenaArgaula-PHT,NaN,NaN,Tomato leaf curl New Delhi begomovirus,coconut flat moth\w*,NaN,NaN
2160,NaN,AgonoxenaArgaula-PHT,NaN,NaN,Tomato leaf curl New Delhi begomovirus,coconut leaf miner\w*,NaN,NaN
2161,NaN,AgonoxenaArgaula-PHT,NaN,NaN,Tomato leaf curl New Delhi begomovirus,papillon\w* des spathes du cocotier,NaN,NaN
2162,NaN,AgonoxenaArgaula-PHT,NaN,NaN,Tomato leaf curl New Delhi begomovirus,polilla plana del coco,NaN,NaN


In [166]:
column_names = ['EPPO_CODE','EFSA_PHT','NCBI_TAXID','name']
column_names = ['a','b','c','name']
df = pd.DataFrame([['x','y',None,'tizio'],[None,'y','z','caio'],['w',None,None,'sempronio']], columns=column_names)
df_out = pd.DataFrame([['x','y','z','tizio'],['x','y','z','caio'],['w',None,None,'sempronio']], columns=column_names)
print('INPUT:')
display(df)
print('The output I want:')
display(df_out)
print('Output of my attempt:')
df.groupby(['a','b','c'], as_index=False).agg({'a' : 'max', 'b' : 'max', 'c' : 'max'})

INPUT:


,a,b,c,name
0,x,y,None,tizio
1,None,y,z,caio
2,w,None,None,sempronio


The output I want:


,a,b,c,name
0,x,y,z,tizio
1,x,y,z,caio
2,w,None,None,sempronio


Output of my attempt:


,a,b,c


In [227]:
column_names = ['EPPO_CODE','EFSA_PHT','NCBI_TAXID','name']
df_sample = pd.DataFrame([['x','y',None,'tizio'],[None,'y','z','caio'],['w',None,None,'sempronio']], columns=column_names)
#display(df_sample)
## Copy NCBI info to all EFSA terms representing the same entity (same EFSA PHT)

cc = ['EPPO_CODE','EFSA_PHT','NCBI_TAXID']
indices = list(range(len(cc)))

#for _ in indices:
for col_i in indices:
    empty_col_indices = indices.copy()
    empty_col_indices.remove(col_i)
    
    ref_col_name = cc[col_i]
    blank_col_names = [cc[x] for x in empty_col_indices] 
#dbg(ref_col_name, '\nNEW ITERATION: ')
    ## Iterate over final db lines 
    for final_db_i in tqdm(range(len(final_db))):
#print('extracting info from row:', final_db_i)
        ## check if row has a value in the reference column 
        ref_col_value = final_db.at[final_db_i,ref_col_name]
#dbg(ref_col_value)
        if ref_col_value:
            ## store id values from other columns
            id_values = {col_name:final_db.at[final_db_i,col_name] for col_name in blank_col_names}
#dbg(id_values, ref_col_value)
#print('all rows with',ref_col_value,'will be completed with',list(id_values.values()))
            for edit_db_i in (range(len(final_db))):
                for col_name in blank_col_names:
#dbg(col_name, 'column')
#dbg(edit_db_i, 'row')
                    if not final_db.at[edit_db_i,col_name] and final_db.at[edit_db_i,ref_col_name]==ref_col_value:
#print('EDITING')
#print(col_name,'none, replace with',id_values[col_name])
                        final_db.at[edit_db_i,col_name] = id_values[col_name]
final_db

100%|██████████| 2164/2164 [00:41<00:00, 51.81it/s] 


,EPPO_CODE,EFSA_PHT,NCBI_TAXID,preferred_name,synonym,re,ds_language,match_language
0,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,ncbi:34020,'Candidatus Liberibacter africanus',greening of citrus,greening of citrus,en,NaN
1,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,ncbi:34020,'Candidatus Liberibacter africanus',greening des agrumes,greening des agrumes,fr,NaN
2,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,ncbi:34020,'Candidatus Liberibacter africanus',virescence des agrumes,vir\wscence des agrumes,fr,NaN
3,LIBEAF,LiberibacterAfricanusAsiaticusAmericanus-PHT,ncbi:34020,'Candidatus Liberibacter africanus',enverdecimiento de los cítricos,enverdecimiento de los c\wtricos,es,NaN
4,LIBEAM,NaN,,'Candidatus Liberibacter americanus',Brazilian citrus greening,NaN,en,NaN
...,...,...,...,...,...,...,...,...
2159,NaN,AgonoxenaArgaula-PHT,NaN,NaN,Tomato leaf curl New Delhi begomovirus,coconut flat moth\w*,NaN,NaN
2160,NaN,AgonoxenaArgaula-PHT,NaN,NaN,Tomato leaf curl New Delhi begomovirus,coconut leaf miner\w*,NaN,NaN
2161,NaN,AgonoxenaArgaula-PHT,NaN,NaN,Tomato leaf curl New Delhi begomovirus,papillon\w* des spathes du cocotier,NaN,NaN
2162,NaN,AgonoxenaArgaula-PHT,NaN,NaN,Tomato leaf curl New Delhi begomovirus,polilla plana del coco,NaN,NaN


In [228]:
final_db.to_csv(LR_filepath)

In [229]:
sci_name = 'Ralstonia pseudosolanacearum'

prep_ncbi_db[(prep_ncbi_db['synonym']=='bacteria') | (prep_ncbi_db['synonym']=="'"+sci_name+"'")]

,NCBI_TAXID,preferred_name,synonym
1,ncbi:2,Bacteria,bacteria
